<a href="https://colab.research.google.com/github/segravjf/python_practice/blob/main/pandas_message_response_time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Time for a response on a messaging application

You are given a dataset with information around messages sent between users in a P2P messaging application. Below is the dataset's schema:



```
Column Name	Data Type	Description
date	string	date of the message sent/received, format is 'YYYY-mm-dd'
timestamp	integer	timestamp of the message sent/received, epoch seconds
sender_id	integer	id of the message sender
receiver_id	integer	id of the message receiver
```

Given this, write code to find the fraction of messages that are sent between the same sender and receiver within five minutes (e.g. the fraction of messages that receive a response within 5 minutes).

In [23]:
## Import modules
import pandas as pd
import numpy as np

In [3]:
## Import data
df = pd.read_csv('https://raw.githubusercontent.com/erood/interviewqs.com_code_snippets/master/Datasets/sample_message_dataset.csv')

df.head()

,date,timestamp,sender_id,receiver_id
0,2018-03-01,1519923378,1,5
1,2018-03-01,1519942810,1,4
2,2018-03-01,1519918950,1,5
3,2018-03-01,1519930114,1,2
4,2018-03-01,1519920410,1,2


In [5]:
## Start by creating a unique conversation id based on sender_id and receiver_id

## First get the lower of the sender_id and receiver_id
df['lower_id'] = df[['sender_id','receiver_id']].min(axis=1)

## Then get teh higher of the two
df['higher_id'] = df[['sender_id','receiver_id']].max(axis=1)

## Then merge them into a composite
df['thread_id'] = df.lower_id.astype(str) + " - " + df.higher_id.astype(str)

df.head()

,date,timestamp,sender_id,receiver_id,lower_id,higher_id,thread_id
0,2018-03-01,1519923378,1,5,1,5,1 - 5
1,2018-03-01,1519942810,1,4,1,4,1 - 4
2,2018-03-01,1519918950,1,5,1,5,1 - 5
3,2018-03-01,1519930114,1,2,1,2,1 - 2
4,2018-03-01,1519920410,1,2,1,2,1 - 2


In [22]:
## Do a lag function on df partitioned by sender_id-receiver_id combo to get the previous message's send time
df['last_message_timestamp'] = df.sort_values(by=['thread_id','timestamp']).groupby(['thread_id'])['timestamp'].shift(1)

## Lag the sender from last time
df['last_sender_id'] = df.sort_values(by=['thread_id','timestamp']).groupby(['thread_id'])['sender_id'].shift(1)

## Subtract current timestamp from previous timestamp to get time difference
df['time_since_last_message'] = df.timestamp - df.last_message_timestamp

## Quick print for sanity
df.sort_values(by=['thread_id','timestamp']).head()

,date,timestamp,sender_id,receiver_id,lower_id,higher_id,thread_id,last_message_timestamp,time_since_last_message,last_sender_id
4,2018-03-01,1519920410,1,2,1,2,1 - 2,NaN,NaN,NaN
47,2018-03-01,1519920555,2,1,1,2,1 - 2,1.519920e+09,145.0,1.0
3,2018-03-01,1519930114,1,2,1,2,1 - 2,1.519921e+09,9559.0,2.0
46,2018-03-01,1519930348,2,1,1,2,1 - 2,1.519930e+09,234.0,1.0
1,2018-03-01,1519942810,1,4,1,4,1 - 4,NaN,NaN,NaN


In [35]:
## Make a flag if the response came within five minutes and the receiver_id of the current thread is the same as the previous sender
# df['five_min_response'] = np.where(df['time_since_last_message'] <= 5*60.0 & df['last_sender_id'].astype(float) == df['receiver_id'],1,0)
cond1 = df['time_since_last_message'] <= 5 * 60.0
cond2 = df['last_sender_id'] == df['receiver_id']

df['five_min_response'] = np.select([cond1 & cond2], [1], default = 0)

df.sort_values(by='thread_id').head()

,date,timestamp,sender_id,receiver_id,lower_id,higher_id,thread_id,last_message_timestamp,time_since_last_message,last_sender_id,five_min_response
3,2018-03-01,1519930114,1,2,1,2,1 - 2,1.519921e+09,9559.0,2.0,0
4,2018-03-01,1519920410,1,2,1,2,1 - 2,NaN,NaN,NaN,0
46,2018-03-01,1519930348,2,1,1,2,1 - 2,1.519930e+09,234.0,1.0,1
47,2018-03-01,1519920555,2,1,1,2,1 - 2,1.519920e+09,145.0,1.0,1
1,2018-03-01,1519942810,1,4,1,4,1 - 4,NaN,NaN,NaN,0


In [38]:
## Spot check some results
df[df['five_min_response'] == 1].sort_values(by='thread_id').head(20)

,date,timestamp,sender_id,receiver_id,lower_id,higher_id,thread_id,last_message_timestamp,time_since_last_message,last_sender_id,five_min_response
46,2018-03-01,1519930348,2,1,1,2,1 - 2,1.519930e+09,234.0,1.0,1
47,2018-03-01,1519920555,2,1,1,2,1 - 2,1.519920e+09,145.0,1.0,1
44,2018-03-01,1519943010,4,1,1,4,1 - 4,1.519943e+09,200.0,1.0,1
56,2018-03-01,1519898323,2,3,2,3,2 - 3,1.519898e+09,211.0,3.0,1
59,2018-03-01,1519901407,2,3,2,3,2 - 3,1.519901e+09,93.0,3.0,1
48,2018-03-01,1519868758,3,2,2,3,2 - 3,1.519869e+09,11.0,2.0,1
53,2018-03-01,1519905977,6,2,2,6,2 - 6,1.519906e+09,8.0,2.0,1
79,2018-03-01,1519928007,2,7,2,7,2 - 7,1.519928e+09,34.0,7.0,1
58,2018-03-01,1519892046,3,3,3,3,3 - 3,1.519892e+09,103.0,3.0,1
57,2018-03-01,1519888329,6,3,3,6,3 - 6,1.519888e+09,160.0,3.0,1


In [40]:
## Now do the calculation for response time

pct_messages_with_response = df.five_min_response.sum() * 1. / df.sender_id.count()

print("Total messages: ", df.sender_id.count())
print("Total responses: ", df.five_min_response.sum())
print("Response rate: ", pct_messages_with_response)

Total messages:  86
Total responses:  21
Response rate:  0.2441860465116279
